In [192]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import skew, kurtosis, jarque_bera
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import PhillipsPerron

In [166]:
df = pd.read_csv('dataset.csv')
df.describe()

,OMR3M,BHD3M,KWD3M,QAR3M,SAR3M,AED3M,EUR3M,EIBO3M,Gold,Oil,MSCI EM,SPX,USDL3M,GAS
count,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000
mean,0.388481,0.380020,0.297868,3.648441,3.752079,3.673406,1.208145,1.845811,1480.835984,71.887241,1020.410507,2532.565324,0.012992,3.429253
std,0.006969,0.002022,0.011122,0.018029,0.003757,0.000974,0.120243,1.319460,300.635917,21.439877,126.902210,1150.320374,0.015958,1.269002
min,0.376000,0.375000,0.273900,3.616850,3.744700,3.664250,0.968440,0.252600,908.500000,-37.630000,688.519000,879.130000,0.001141,1.190500
25%,0.384060,0.378420,0.290275,3.642000,3.750025,3.672875,1.109440,0.828860,1241.950000,53.270000,952.120250,1512.382500,0.002681,2.621000
50%,0.386580,0.379790,0.298035,3.643600,3.750750,3.673312,1.178455,1.513330,1366.500000,72.640000,1005.627000,2186.320000,0.004679,3.061500
75%,0.390977,0.381650,0.307685,3.648548,3.753300,3.674000,1.312795,2.342500,1746.187500,89.490000,1079.393250,3344.577500,0.018944,4.022250
max,0.435130,0.386150,0.323950,3.864650,3.778050,3.677300,1.508790,5.602640,2401.500000,123.700000,1444.930000,5254.350000,0.056874,9.680000


In [167]:
df.loc[df['Oil'] < 0]

,Date,OMR3M,BHD3M,KWD3M,QAR3M,SAR3M,AED3M,EUR3M,EIBO3M,Gold,Oil,MSCI EM,SPX,USDL3M,GAS
2852,2020/4/20,0.42025,0.37705,0.3179,3.6542,3.75725,3.674875,1.09061,1.57708,1686.2,-37.63,899.326,2823.16,0.010976,1.924


In [168]:
df.iloc[[2851,2852,2853],10]

2851    18.27
2852   -37.63
2853     8.91
Name: Oil, dtype: float64

In [169]:
rate2852 = (df.iloc[2852,10] - df.iloc[2851,10]) / df.iloc[2851,10]*100
rate2853 = (df.iloc[2853,10] - df.iloc[2852,10]) / df.iloc[2852,10]*-100

In [170]:
df['Date'] = pd.to_datetime(df['Date'])
for column in df.columns.tolist()[1:]:
    df[column] = (np.log(df[column]) - np.log(df[column].shift(1)))*100
df.iloc[2852, 10] = rate2852
df.iloc[2853, 10] = rate2853
df = df.iloc[1:,:]
df.isna().sum()

c:\Users\entre\anaconda3\envs\ML\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Date       0
OMR3M      0
BHD3M      0
KWD3M      0
QAR3M      0
SAR3M      0
AED3M      0
EUR3M      0
EIBO3M     0
Gold       0
Oil        0
MSCI EM    0
SPX        0
USDL3M     0
GAS        0
dtype: int64

In [171]:
df.columns.tolist()[1:]

['OMR3M',
 'BHD3M',
 'KWD3M',
 'QAR3M',
 'SAR3M',
 'AED3M',
 'EUR3M',
 'EIBO3M',
 'Gold',
 'Oil',
 'MSCI EM',
 'SPX',
 'USDL3M',
 'GAS']

In [179]:
columns=['Name', 'Mean', 'Maximun', 'Minimun', 'Std. Dev.', 'Skewness', 'Kurtosis', 'Jarque-Bera', 'ADF', 'PP']
ls = []
for i, column in enumerate(df.columns.tolist()[1:]):
    ls.append([column, df[column].mean(), df[column].max(), df[column].min(), df[column].std(), skew(df[column]), kurtosis(df[column]), jarque_bera(df[column])[1], adfuller(df[column])[1], PhillipsPerron(df[column]).pvalue])
df_stats = pd.DataFrame.from_records(ls, columns=columns)

In [189]:
cor_matrix = df.iloc[:,1:].corr()
print(cor_matrix)

            OMR3M     BHD3M     KWD3M     QAR3M     SAR3M     AED3M     EUR3M  \
OMR3M    1.000000  0.064175 -0.004342 -0.032026  0.075297  0.073529  0.007711   
BHD3M    0.064175  1.000000  0.048295  0.014369  0.038707  0.018838  0.010918   
KWD3M   -0.004342  0.048295  1.000000  0.021553  0.016946  0.087568 -0.202587   
QAR3M   -0.032026  0.014369  0.021553  1.000000  0.024471  0.002695  0.010328   
SAR3M    0.075297  0.038707  0.016946  0.024471  1.000000  0.064088  0.000726   
AED3M    0.073529  0.018838  0.087568  0.002695  0.064088  1.000000 -0.025512   
EUR3M    0.007711  0.010918 -0.202587  0.010328  0.000726 -0.025512  1.000000   
EIBO3M  -0.011979 -0.014450 -0.010113 -0.013793  0.013951 -0.004707  0.021393   
Gold     0.037293  0.004764 -0.084953  0.002930  0.003426 -0.016769  0.363884   
Oil      0.058072  0.036930 -0.018069 -0.002890  0.082453  0.014696  0.049036   
MSCI EM -0.000011 -0.000820 -0.064095 -0.016388 -0.003269  0.012727  0.292853   
SPX      0.047305 -0.009077 

In [188]:
latex_output = df_stats.to_latex(index=False)
# Write the LaTeX output to a file
with open('summary.tex', 'w') as f:
    f.write(latex_output)

In [191]:
latex_output = cor_matrix.to_latex()
# Write the LaTeX output to a file
with open('cor.tex', 'w') as f:
    f.write(latex_output)